In [1]:
from bs4 import BeautifulSoup
import urllib
import requests
import urllib
import pandas as pd
import time
import ftfy
import random
from fake_useragent import UserAgent

In [2]:
classes = pd.read_csv("iab v2 keywords.csv", engine = "python").fillna("NA")
classes = classes.rename(columns = classes.iloc[4])[5:].reset_index()
classes = classes[['Unique ID ', 'Parent', 'Name']]
classes = classes[:200]
ua = UserAgent()

In [3]:
classes

,Unique ID,Parent,Name
0,1,NA,Automotive
1,2,1,Auto Body Styles
2,3,2,Commercial Trucks
3,4,2,Sedan
4,5,2,Station Wagon
...,...,...,...
195,196,192,Parenting Babies and Toddlers
196,197,192,Parenting Children Aged 4-11
197,198,192,Parenting Teens
198,199,192,Special Needs Kids


In [4]:
def apply(input):
    query = input
    address = "http://www.bing.com/search?q=%s" % (urllib.parse.quote_plus(query)) # https://docs.python.org/3/library/urllib.parse.html
    #address = "http://www.bing.com/search?q=%s" % (urllib.quote_plus(query))
    address += "&first=10"
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'}
    r = requests.get(address, headers = headers)
    print(r.status_code)
    html_response = r.text
    return_dict = {}
    return_dict["status_code"] = r.status_code

    if return_dict["status_code"] == 200:
        soup = BeautifulSoup(html_response, 'html.parser')
        [s.extract() for s in soup('span')]
        results = soup.findAll('li', { "class" : "b_algo" })
        temp_list = []
        i = 1
        for result in results:
            temp_dict = {}
            intermediate_result = result.find('h2')
            temp_dict["position"] = str(i)
            try:
                temp_dict["url"] = intermediate_result.find('a')['href']
            except Exception as E:
                temp_dict["url"] = ''
    
            try:
                temp_dict["title"] = intermediate_result.get_text()
            except Exception as E:
                temp_dict["title"] = ''
    
            try:
                temp_dict["snippet"] = result.find('p').get_text()
            except Exception as E:
                temp_dict["snippet"] = ''
    
            temp_list.append(temp_dict)
            #print(intermediate_result.get_text())
            #print(intermediate_result.find('a')['href'])
    
            #print(result.find('p').get_text())
            #print("*"*20)
            i = i+1
        return_dict["Results"] = temp_list
    #print(return_dict)

#     print(address)
    return return_dict

In [5]:
data = pd.DataFrame(columns = ['Unique ID', 'Parent', 'Name', 'Position', 'URL', 'Title', 'Snippet'])

In [6]:
for i in range(len(classes)):
    time.sleep(random.randint(45, 60))
    res = apply(classes.iloc[i].Name)
    while res['status_code'] != 200 or len(res['Results']) < 8:
        print(f"Found {len(res['Results'])}, trying again")
        time.sleep(random.randint(20, 30))
        res = apply(classes.iloc[i].Name)
    print(f"Found {len(res['Results'])} for {classes.iloc[i].Name}")
    for j in res['Results']:
        row = pd.Series(list([classes.iloc[i]['Unique ID '], classes.iloc[i]['Parent'], classes.iloc[i]['Name'], j['position'], j['url'], j['title'], ftfy.fix_text(j['snippet'])]), index=data.columns)
        data = data.append(row, ignore_index=True)   
    print()

200
Found 10 for Automotive

200
Found 10 for Auto Body Styles

200
Found 10 for Commercial Trucks

200
Found 10 for Sedan

200
Found 10 for Station Wagon

200
Found 10 for SUV

200
Found 10 for Van

200
Found 10 for Convertible

200
Found 10 for Coupe

200
Found 10 for Crossover

200
Found 10 for Hatchback

200
Found 10 for Microcar

200
Found 10 for Minivan

200
Found 10 for Off-Road Vehicles

200
Found 10 for Pickup Trucks

200
Found 10 for Auto Type

200
Found 10 for Budget Cars

200
Found 10 for Certified Pre-Owned Cars

200
Found 10 for Classic Cars

200
Found 10 for Concept Cars

200
Found 10 for Driverless Cars

200
Found 10 for Green Vehicles

200
Found 10 for Luxury Cars

200
Found 10 for Performance Cars

200
Found 10 for Car Culture

200
Found 10 for Dash Cam Videos

200
Found 10 for Motorcycles

200
Found 10 for Road-Side Assistance

200
Found 10 for Scooters

200
Found 10 for Auto Buying and Selling

200
Found 10 for Auto Insurance

200
Found 10 for Auto Parts

200
Found 

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [8]:
data.Name.value_counts()

Business Operations               10
Human Resources                   10
Motorcycles                       10
Sale & Lease Back                 10
Auto Repair                       10
                                  ..
Auto Safety Technologies          10
Road-Side Assistance              10
Auto Infotainment Technologies    10
Venture Capital                   10
Classic Cars                      10
Name: Name, Length: 77, dtype: int64

In [9]:
data.tail(11)

,Unique ID,Parent,Name,Position,URL,Title,Snippet
759,76,53,Executive Leadership & Management,10,https://bluenotes.anz.com/topics/leadership-an...,Leadership & Management,"· Subscribe. Something goes wrong, please ref..."
760,77,53,Government Business,1,https://msme.gov.in/,"Ministry of Micro, Small & Medium Enterprises","Ministry of Micro, Small & Medium Enterprises ..."
761,77,53,Government Business,2,https://www.startupindia.gov.in/,Homepage [www.startupindia.gov.in],Startup India is a flagship initiative of the ...
762,77,53,Government Business,3,https://accountlearning.com/top-10-responsibil...,Top 10 Responsibilities of Government towards ...,Government agencies publish and provide a larg...
763,77,53,Government Business,4,https://www.bajajfinserv.in/insights/governmen...,Government Loan Schemes for Small Businesses i...,Credit guarantee fund government business loan...
764,77,53,Government Business,5,https://en.wikipedia.org/wiki/Business.gov,Business.gov - Wikipedia,"Business.gov includes federal, state and local..."
765,77,53,Government Business,6,https://managementmania.com/en/g2b-government-...,G2B (Government to Business) - ManagementMania...,G2B (Government to Business) is a term that re...
766,77,53,Government Business,7,https://www.india.gov.in/my-government/schemes,Schemes | National Portal of India - Governmen...,"Indian Government, at all levels, announces We..."
767,77,53,Government Business,8,https://www.usa.gov/business,Small Business | USAGov,Search handpicked government websites helpful ...
768,77,53,Government Business,9,https://www.business.govt.nz/,Home — business.govt.nz,COVID-19 Government info for businesses. Find ...


In [16]:
classes[classes.Name == 'Government Business']

,Unique ID,Parent,Name
76,77,53,Government Business


In [18]:
classes.iloc[76]

Unique ID                      77
Parent                         53
Name          Government Business
Name: 76, dtype: object

In [19]:
classes.tail()

,Unique ID,Parent,Name
195,196,192,Parenting Babies and Toddlers
196,197,192,Parenting Children Aged 4-11
197,198,192,Parenting Teens
198,199,192,Special Needs Kids
199,200,186,Single Life


In [20]:
for i in range(77, 200):
    time.sleep(random.randint(45, 60))
    res = apply(classes.iloc[i].Name)
    while res['status_code'] != 200 or len(res['Results']) < 8:
        print(f"Found {len(res['Results'])}, trying again")
        time.sleep(random.randint(20, 30))
        res = apply(classes.iloc[i].Name)
    print(f"Found {len(res['Results'])} for {classes.iloc[i].Name}")
    for j in res['Results']:
        row = pd.Series(list([classes.iloc[i]['Unique ID '], classes.iloc[i]['Parent'], classes.iloc[i]['Name'], j['position'], j['url'], j['title'], ftfy.fix_text(j['snippet'])]), index=data.columns)
        data = data.append(row, ignore_index=True)   
    print()

200
Found 10 for Green Solutions

200
Found 10 for Business Utilities

200
Found 10 for Economy

200
Found 10 for Commodities

200
Found 10 for Currencies

200
Found 10 for Financial Crisis

200
Found 10 for Financial Reform

200
Found 10 for Financial Regulation

200
Found 10 for Gasoline Prices

200
Found 10 for Housing Market

200
Found 10 for Interest Rates

200
Found 10 for Job Market

200
Found 10 for Industries

200
Found 10 for Advertising Industry

200
Found 10 for Education industry

200
Found 10 for Entertainment Industry

200
Found 10 for Environmental Services Industry

200
Found 10 for Financial Industry

200
Found 10 for Food Industry

200
Found 10 for Healthcare Industry

200
Found 10 for Hospitality Industry

200
Found 10 for Information Services Industry

200
Found 10 for Legal Services Industry

200
Found 10 for Logistics and Transportation Industry

200
Found 10 for Agriculture

200
Found 10 for Management Consulting Industry

200
Found 10 for Manufacturing Industry

In [21]:
data.Name.value_counts()

Dating                           10
Travel Books                     10
Auto Buying and Selling          10
Primary Education                10
Poetry                           10
                                 ..
Consumer Issues                  10
Minivan                          10
Parenting Babies and Toddlers    10
Food Industry                    10
Single Life                      10
Name: Name, Length: 200, dtype: int64

In [22]:
set(classes.Name) == set(data.Name.unique())

True

In [23]:
data

,Unique ID,Parent,Name,Position,URL,Title,Snippet
0,1,NA,Automotive,1,https://www.merriam-webster.com/dictionary/aut...,Automotive | Definition of Automotive by …,Automotive definition is - self-propelled. How...
1,1,NA,Automotive,2,https://nl.wikipedia.org/wiki/Automotive,Automotive - Wikipedia,"Dit is een doorverwijspagina, bedoeld om de ve..."
2,1,NA,Automotive,3,https://www.dictionary.com/browse/automotive,Automotive | Definition of Automotive at …,"Automotive definition, pertaining to the desig..."
3,1,NA,Automotive,4,https://en.wikipedia.org/wiki/Automotive_industry,Automotive industry - Wikipedia,The automotive industry comprises a wide range...
4,1,NA,Automotive,5,https://automotive-online.nl/management/laatst...,Alle nieuwsrubrieken - Automotive Online,Het platform voor het midden en hoger manageme...
...,...,...,...,...,...,...,...
1995,200,186,Single Life,6,https://singlelife.today/,Single Life Today,Single Life Today. MEMBER LOGIN. COME JOIN THE...
1996,200,186,Single Life,7,https://www.nickiswift.com/319063/the-untold-t...,The Untold Truth Of 90 Day: The Single Life,· While the original 90 Day Fiancé follows co...
1997,200,186,Single Life,8,https://www.newsnow.co.uk/h/?search=%22Single+...,"NewsNow: ""Single Life"" headlines | Every Sourc...",90 Day The Single Life: TLC Unveils Brand New ...
1998,200,186,Single Life,9,https://www.elitesingles.ca/en/mag/single-life...,Single Life | Page 7 of 12 | EliteSingles,Single Life 28.08.2017 Taking the rough with t...


In [24]:
data.to_csv('IAB_text.csv', mode = 'a')

In [25]:
df = pd.read_csv("IAB_text.csv")

In [27]:
df.Name.value_counts()

Antarctica             30
Tonga                  20
Email                  20
Cheerleading           10
Akan                   10
                       ..
Home Improvement       10
Album-oriented Rock    10
Saint Lucia            10
Medical Tests          10
Search                  8
Name: Name, Length: 1192, dtype: int64

In [28]:
classes = pd.read_csv("iab v2 keywords.csv", engine = "python").fillna("NA")
classes = classes.rename(columns = classes.iloc[4])[5:].reset_index()
classes = classes[['Unique ID ', 'Parent', 'Name']]

In [29]:
set(df.Name.unique()) == set(classes.Name.unique())

False

In [30]:
classes.Name.unique()

array(['Automotive', 'Auto Body Styles', 'Commercial Trucks', ...,
       'Region/State', 'Metro', 'City'], dtype=object)

In [31]:
df.Name.unique()

array(['Automotive', 'Auto Body Styles', 'Commercial Trucks', ...,
       'Region/State', 'Metro', 'City'], dtype=object)

In [33]:
set(df.Name.unique()).difference(set(classes.Name.unique()))

{nan}